In [1]:
import os
import numpy as np
import progressbar
from sklearn.model_selection import KFold
import math

# Loading the pairs

Getting the same pairs, stored on `pairs.txt`, as the validation on the [Facenet](https://github.com/davidsandberg/facenet). 

The `pairs.txt` file is divided between two use cases:
* Lines with one *name* and two *numbers* means positive samples, i.e., we will compare two different images of the same user on the test set.
* Lines with two *names* and two *numbers* means negative samples, i.e., we will compare two different photos, composed by the name of the first user and number of picture of the first user vs. the name of second user and number of the second user picture.

This information is used to prepare the **helper variables**, that will be extremely important to run the validation.

## Helper functions

In [16]:
def read_pairs(path):
    pairs=[]
    with open(path, 'r') as f:
        for line in f.readlines()[1:]:
            pair = line.strip().split()
            pairs.append(pair)
            
    return np.array(pairs)

In [17]:
def create_path(lfw_dir, pair, output):
    if len(pair) == 3:
        # Use case -- same
        path0 = os.path.join(lfw_dir, pair[0], output, pair[0] + '_' + '%04d' % int(pair[1])) + '.npy'
        path1 = os.path.join(lfw_dir, pair[0], output, pair[0] + '_' + '%04d' % int(pair[2])) + '.npy'
        issame = True
    elif len(pair) == 4:
        # Use case -- differente
        path0 = os.path.join(lfw_dir, pair[0], output, pair[0] + '_' + '%04d' % int(pair[1])) + '.npy'
        path1 = os.path.join(lfw_dir, pair[2], output, pair[2] + '_' + '%04d' % int(pair[3])) + '.npy'
        issame = False
    return path0, path1, issame

In [18]:
def get_paths(lfw_dir, pairs, output):
    nrof_skipped_pairs = 0
    path_list = []
    issame_list = []
    
    for pair in pairs:
        path0, path1, issame = create_path(lfw_dir, pair, output)
            
        if os.path.exists(path0) and os.path.exists(path1):
            path_list += (path0, path1)
            issame_list.append(issame)
        else:
            nrof_skipped_pairs += 1
            
    if nrof_skipped_pairs > 0:
        print('There was %d skipped pairs.' % nrof_skipped_pairs)
        
    return path_list, issame_list

## Running the code

Running the code from the *helper functions* to obtain the **global variables** which will manage the entire **validation** session.

In [19]:
# Define the global variables, used over this code
pairs_path = '/Users/pedroprates/Google Drive/FaceRecognition/data/pairs.txt'
lfw_path = '/Users/pedroprates/Google Drive/FaceRecognition/datasets/lfw/lfw_mtcnnpy_160'

In [20]:
pairs = read_pairs(pairs_path)
print("pairs.txt was successfully read and it was retrieved a ndarray with shape", str(pairs.shape))

pairs.txt was successfully read and it was retrieved a ndarray with shape (6000,)


In [24]:
path_list, actual_issame = get_paths(lfw_path, pairs, 'mobilenetv2_v1')

# Running the validation

Now we are going to run the validation based on the `npy` paths that we've acquired.


In [8]:
def distance(embeddings1, embeddings2, distance_metric='euclidean'):
    """ Calculate the distance between two embeddings. Currently working with euclidean and cosine similarity. 

        :param embeddings1: First embedding
        :param embeddings2: Second embedding
        :param distance_metric: Distance metric to be used to make the calculation. Should be either: 'euclidean' or 'cosine'

        :returns: The distance between the `embeddings1` and `embeddings2`
    """
    assert distance_metric in ['euclidean', 'cosine'], "The distance metric should be either 'euclidean' or 'cosine'"

    if distance_metric == 'euclidean':
        diff = np.subtract(embeddings1, embeddings2)
        dist = np.sum(np.square(diff), 1)

    elif distance_metric == 'cosine':
        dot = np.sum(np.multiply(embeddings1, embeddings2), axis=1)
        norm = np.linalg.norm(embeddings1, axis=1) * np.linalg.norm(embeddings2, axis=1)
        similarity = dot / norm
        dist = np.arccos(similarity) / math.pi

    return dist

In [9]:
def load_embeddings(paths):
    nrof_skips = 0
    bt_size = len(paths)
    embeddings = np.zeros((bt_size, 512))
    
    for i, path in enumerate(paths):
        if not os.path.exists(path):
            nrof_skips += 1
            continue
        emb = np.load(path)
        embeddings[i, :] = emb
        
    if nrof_skips > 0:
        print("There are %d files that weren't found" % nrof_skips)
        
    return embeddings

In [10]:
def calculate_accuracy(threshold, dist, actual_issame):
    predict_issame = np.less(dist, threshold)
    tp = np.sum(np.logical_and(predict_issame, actual_issame))
    fp = np.sum(np.logical_and(predict_issame, np.logical_not(actual_issame)))
    tn = np.sum(np.logical_and(np.logical_not(predict_issame), np.logical_not(actual_issame)))
    fn = np.sum(np.logical_and(np.logical_not(predict_issame), actual_issame))
    
    tpr = 0 if tp + fn == 0 else float(tp) / (tp + fn)
    fpr = 0 if fp + tn == 0 else float(fp) / (fp + tn)
    acc = float(tp+tn) / (tp+fp+tn+fn)
    
    return tpr, fpr, acc

In [11]:
def calculate_roc(thresholds,
                  embeddings1,
                  embeddings2,
                  actual_issame,
                  distance_metric='cosine',
                  subtract_mean=True,
                  nrof_folds=10):

    assert embeddings1.shape[0] == embeddings2.shape[0]
    assert embeddings1.shape[1] == embeddings2.shape[1]

    kfolds = KFold(n_splits=nrof_folds, shuffle=False)

    nrof_pairs = min(len(actual_issame), embeddings1.shape[0])
    nrof_thresholds = len(thresholds)

    tprs = np.zeros((nrof_folds, nrof_thresholds))
    fprs = np.zeros((nrof_folds, nrof_thresholds))
    accuracy = np.zeros(nrof_folds)

    indices = np.arange(nrof_pairs)

    for fold_idx, (train_set, test_set) in enumerate(kfolds.split(indices)):
        if subtract_mean:
            mean = np.mean(np.concatenate([embeddings1[train_set], embeddings2[train_set]], axis=0))
        else:
            mean = 0

        dist = distance(embeddings1-mean, embeddings2-mean, distance_metric)
        acc_train = np.zeros(nrof_thresholds)

        for threshold_idx, threshold in enumerate(thresholds):
            _, _, acc_train[threshold_idx] = calculate_accuracy(threshold, dist[train_set], actual_issame[train_set])
        best_threshold_idx = np.argmax(acc_train)

        for threshold_idx, threshold in enumerate(thresholds):
            tprs[fold_idx, threshold_idx], fprs[fold_idx, threshold_idx], _ = calculate_accuracy(threshold,
                                                                                                 dist[test_set],
                                                                                                 actual_issame[test_set])
        _, _, accuracy[fold_idx] = calculate_accuracy(thresholds[best_threshold_idx],
                                                      dist[test_set],
                                                      actual_issame[test_set])

        tpr = np.mean(tprs, 0)
        fpr = np.mean(fprs, 0)

    return tpr, fpr, accuracy

In [12]:
def evaluate(embeddings, actual_issame, distance_metric='cosine', subtract_mean=False):
    thresholds = np.arange(0, 4, 0.01)
    embeddings1 = embeddings[0::2]
    embeddings2 = embeddings[1::2]
    tpr, fpr, acc = calculate_roc(thresholds, 
                                  embeddings1, 
                                  embeddings2, 
                                  np.array(actual_issame), 
                                  distance_metric=distance_metric, 
                                  subtract_mean=subtract_mean,
                                  nrof_folds=10)
    return tpr, fpr, acc

In [25]:
embeddings = load_embeddings(path_list)

In [26]:
tpr, fpr, acc_mobile = evaluate(embeddings, actual_issame, subtract_mean=True)

In [27]:
acc_mobile

array([0.60166667, 0.56166667, 0.56333333, 0.54      , 0.57      ,
       0.57666667, 0.565     , 0.535     , 0.55166667, 0.53333333])

In [15]:
acc_resnet

array([0.99      , 0.97833333, 0.97333333, 0.98      , 0.99      ,
       0.98833333, 0.98      , 0.99166667, 0.98833333, 0.99166667])

# Running Forward Propagation

In [76]:
import tensorflow as tf
import src.facenet as facenet
from scipy import misc

In [60]:
def process_image(path):
    original_image = misc.imread(path)
    image = original_image.reshape((1, *original_image.shape))

    return image

In [58]:
def forward_propagation(image, sess):
    images_placeholder = tf.get_default_graph().get_tensor_by_name("input:0")
    embeddings = tf.get_default_graph().get_tensor_by_name("embeddings:0")
    phase_train_placeholder = tf.get_default_graph().get_tensor_by_name("phase_train:0")

    # Get embeddings
    embeddings_size = embeddings.get_shape()[1]
    feed_dict = { images_placeholder: image, phase_train_placeholder: False }
    embeddings_array = np.zeros((1, embeddings_size))
    embeddings_array = sess.run(embeddings, feed_dict=feed_dict)

    return embeddings_array

In [77]:
def get_embeddings(paths):
    nrof_skips = 0
    bt_size = len(paths)
    embeddings = np.zeros((bt_size, 512))
    sess=tf.Session()
    facenet.load_model('/Users/pedroprates/Google Drive/FaceRecognition/models/facenet/20180402-114759/20180402-114759.pb')
    
    for i, path in progressbar.progressbar(enumerate(paths)):
        if not os.path.exists(path):
            nrof_skips += 1
            continue
        
        image = process_image(path)
        emb = forward_propagation(image, sess)
        embeddings[i, :] = emb
        
    if nrof_skips > 0:
        print("There are %d files that weren't found" % nrof_skips)
        
    return embeddings

In [64]:
def get_paths(lfw_dir, pairs):
    nrof_skipped_pairs = 0
    path_list = []
    issame_list = []
    for pair in pairs:
        if len(pair) == 3:
            path0 = add_extension(os.path.join(lfw_dir, pair[0], pair[0] + '_' + '%04d' % int(pair[1])))
            path1 = add_extension(os.path.join(lfw_dir, pair[0], pair[0] + '_' + '%04d' % int(pair[2])))
            issame = True
        elif len(pair) == 4:
            path0 = add_extension(os.path.join(lfw_dir, pair[0], pair[0] + '_' + '%04d' % int(pair[1])))
            path1 = add_extension(os.path.join(lfw_dir, pair[2], pair[2] + '_' + '%04d' % int(pair[3])))
            issame = False
        if os.path.exists(path0) and os.path.exists(path1):    # Only add the pair if both paths exist
            path_list += (path0,path1)
            issame_list.append(issame)
        else:
            nrof_skipped_pairs += 1
    if nrof_skipped_pairs>0:
        print('Skipped %d image pairs' % nrof_skipped_pairs)
    
    return path_list, issame_list


def add_extension(path):
    if os.path.exists(path+'.jpg'):
        return path+'.jpg'
    elif os.path.exists(path+'.png'):
        return path+'.png'
    else:
        raise RuntimeError('No file "%s" with extension png or jpg.' % path)

In [65]:
# Define the global variables, used over this code
pairs_path = '/Users/pedroprates/Google Drive/FaceRecognition/data/pairs.txt'
lfw_path = '/Users/pedroprates/Google Drive/FaceRecognition/datasets/lfw/lfw_mtcnnpy_160'

In [67]:
pairs = read_pairs(pairs_path)

In [69]:
path_list, actual_issame = get_paths(lfw_path, pairs)

In [111]:
embs = np.load('/Users/pedroprates/Google Drive/FaceRecognition/datasets/lfw_from_facenet.npy')

In [112]:
embs.shape

(12000, 512)

In [78]:
embeddings = get_embeddings(path_list)

Model filename: /Users/pedroprates/Google Drive/FaceRecognition/models/facenet/20180402-114759/20180402-114759.pb


/ |#                                                  | 0 Elapsed Time: 0:00:00/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  
| |                               #               | 11999 Elapsed Time: 0:16:05


In [113]:
tpr, fpr, acc = evaluate(embs, actual_issame, subtract_mean=True)

In [114]:
acc

array([0.98666667, 0.985     , 0.97833333, 0.98666667, 0.99      ,
       0.99166667, 0.98666667, 0.99333333, 0.99333333, 0.99166667])

In [86]:
def calculate_accuracy(threshold, dist, actual_issame):
    predict_issame = np.less(dist, threshold)
    tp = np.sum(np.logical_and(predict_issame, actual_issame))
    fp = np.sum(np.logical_and(predict_issame, np.logical_not(actual_issame)))
    tn = np.sum(np.logical_and(np.logical_not(predict_issame), np.logical_not(actual_issame)))
    fn = np.sum(np.logical_and(np.logical_not(predict_issame), actual_issame))
  
    tpr = 0 if (tp+fn==0) else float(tp) / float(tp+fn)
    fpr = 0 if (fp+tn==0) else float(fp) / float(fp+tn)
    acc = float(tp+tn)/dist.size
    return tpr, fpr, acc

In [96]:
def calculate_roc(thresholds, embeddings1, embeddings2, actual_issame, nrof_folds=10, distance_metric=0, subtract_mean=False):
    assert(embeddings1.shape[0] == embeddings2.shape[0])
    assert(embeddings1.shape[1] == embeddings2.shape[1])
    nrof_pairs = min(len(actual_issame), embeddings1.shape[0])
    nrof_thresholds = len(thresholds)
    k_fold = KFold(n_splits=nrof_folds, shuffle=False)
    
    tprs = np.zeros((nrof_folds,nrof_thresholds))
    fprs = np.zeros((nrof_folds,nrof_thresholds))
    accuracy = np.zeros((nrof_folds))
    
    indices = np.arange(nrof_pairs)
    
    for fold_idx, (train_set, test_set) in enumerate(k_fold.split(indices)):
        if subtract_mean:
            mean = np.mean(np.concatenate([embeddings1[train_set], embeddings2[train_set]]), axis=0)
        else:
            mean = 0.0
        dist = distance(embeddings1-mean, embeddings2-mean, distance_metric)
        
        # Find the best threshold for the fold
        acc_train = np.zeros((nrof_thresholds))
        for threshold_idx, threshold in enumerate(thresholds):
            _, _, acc_train[threshold_idx] = calculate_accuracy(threshold, dist[train_set], actual_issame[train_set])
        best_threshold_index = np.argmax(acc_train)
        for threshold_idx, threshold in enumerate(thresholds):
            tprs[fold_idx,threshold_idx], fprs[fold_idx,threshold_idx], _ = calculate_accuracy(threshold, dist[test_set], actual_issame[test_set])
        _, _, accuracy[fold_idx] = calculate_accuracy(thresholds[best_threshold_index], dist[test_set], actual_issame[test_set])
          
        tpr = np.mean(tprs,0)
        fpr = np.mean(fprs,0)
    return tpr, fpr, accuracy

In [115]:
def evaluate(embeddings, actual_issame, nrof_folds=10, distance_metric='cosine', subtract_mean=False):
    # Calculate evaluation metrics
    thresholds = np.arange(0, 4, 0.01)
    embeddings1 = embeddings[0::2]
    embeddings2 = embeddings[1::2]
    tpr, fpr, accuracy = calculate_roc(thresholds, embeddings1, embeddings2,
        np.asarray(actual_issame), nrof_folds=nrof_folds, distance_metric=distance_metric, subtract_mean=subtract_mean)
    thresholds = np.arange(0, 4, 0.001)
#     val, val_std, far = facenet.calculate_val(thresholds, embeddings1, embeddings2,
#         np.asarray(actual_issame), 1e-3, nrof_folds=nrof_folds, distance_metric=distance_metric, subtract_mean=subtract_mean)
    return tpr, fpr, accuracy

In [123]:
def get_image_number(imagename):
    imagename = imagename.split('.')[0]
    imagename = imagename.split('_')[-1]
    
    return int(imagename)

In [128]:
lfw_dir = '/Users/pedroprates/Google Drive/FaceRecognition/datasets/lfw/lfw_mtcnnpy_160/'
people = os.listdir(lfw_dir)
people = list(filter(lambda x: os.path.isdir(os.path.join(lfw_dir, x)), people))

images = []
for person in progressbar.progressbar(people):
    person_folder = os.path.join(lfw_dir, person)
    list_pics = os.listdir(person_folder)
    list_pics = list(filter(lambda x: x.endswith('.png') or x.endswith('.jpg'), list_pics))
    
    for pic in list_pics:
        img_number = get_image_number(pic)
        images += (person, img_number)

100% (5749 of 5749) |####################| Elapsed Time: 0:00:00 Time:  0:00:00


In [139]:
output_dir = '/Users/pedroprates/Google Drive/FaceRecognition/data/all_dataset.txt'
with open(output_dir, 'w') as f:
    for i in range(0, len(images), 4):
        to_write = images[i] + '\t' + str(images[i+1]) + '\t' 
        if (i+2) >= len(images):
            to_write += images[i] + '\t' + str(images[i+1])
        else:
            to_write += images[i+2] + '\t' + str(images[i+3])
            
        to_write += '\n'
        f.write(to_write)

In [142]:
len(images) // 2

13233

In [144]:
embs = np.load('/Users/pedroprates/Google Drive/FaceRecognition/datasets/all_lfw.npy')

In [146]:
embs.shape

(13234, 512)

In [147]:
pairs_path = '/Users/pedroprates/Google Drive/FaceRecognition/data/all_dataset.txt'
pairs = read_pairs(pairs_path)
path_list, actual_issame = get_paths(lfw_path, pairs)

In [148]:
len(path_list)

13234

In [150]:
path_list[0]

'/Users/pedroprates/Google Drive/FaceRecognition/datasets/lfw/lfw_mtcnnpy_160/German_Khan/German_Khan_0001.png'

In [152]:
change_path(path_list[0])

'/Users/pedroprates/Google Drive/FaceRecognition/datasets/lfw/lfw_mtcnnpy_160/German_Khan/output_resnet/German_Khan_0001.npy'

In [154]:
def change_path(path, lfw_dir='/Users/pedroprates/Google Drive/FaceRecognition/datasets/lfw/lfw_mtcnnpy_160'):
    path = path.split('/')
    name = path[-2]
    file = path[-1]
    
    file = file.split('.')[0] + '.npy'
    
    return os.path.join(lfw_dir, name, 'output_resnet', file), os.path.join(lfw_dir, name, 'output_resnet')

In [156]:
for index, embedding in progressbar.progressbar(enumerate(embs)):
    embedding = embedding.reshape(1, *embedding.shape)
    
    save_path, folder_path = change_path(path_list[index])
    if not os.path.exists(folder_path):
        os.mkdir(folder_path)
        
    np.save(save_path, embedding)

| |  #                                            | 13233 Elapsed Time: 0:00:19
